In [71]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd
import pyrr
from tqdm import tqdm

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
# import lad
import figures

# from dotenv import load_dotenv
# load_dotenv()

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Params init

Initialize parameters

In [6]:
mockname = 'test'

## Prepare data

Convert Blensor output txt files that have fake numpy extension to real npy

In [7]:
loads.numpy2npy(mockname)

Next wee create the module to segmentate trees. This will be tuned acordingly for each data set.

In [8]:
def segtree(df, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


In [9]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=False)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df)

s0600000.npy (22500, 16) (22500, 3)
s0700000.npy (22500, 16) (22500, 3)
s0100000.npy (22500, 16) (22500, 3)
s0500000.npy (22500, 16) (22500, 3)
s0400000.npy (22500, 16) (22500, 3)
s0200000.npy (22500, 16) (22500, 3)
s0300000.npy (22500, 16) (22500, 3)


In [10]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
loads.showPCfromDF(df[keep])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


## Compute the `Leaf Inclination Angle` (LIA) for each tree

In [18]:
for key, val in trees.items():

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)
    res = lia.bestfit_pars_la(points, mockname, treename=key, weigths=True)
    lia.best_fit_pars_plot(res, key, mockname)


voxel_size_w 0.0001 DONE...
voxel_size_w 0.001 DONE...
voxel_size_w 0.01 DONE...
voxel_size_w 0.1 DONE...
voxel_size_w 1 DONE...
voxel_size_w BESTFIT:	 0.01
kd3_sr 0.001 DONE...
kd3_sr 0.01 DONE...
kd3_sr 0.1 DONE...
kd3_sr 1.0 DONE...
kd3_sr BESTFIT:	 1.0
max_nn 3 DONE...
max_nn 5 DONE...
max_nn 10 DONE...
max_nn 20 DONE...
max_nn 50 DONE...
max_nn 100 DONE...
max_nn BESTFIT:	 5


In [22]:
# load bestfit results
for key, val in trees.items():

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)

    bestfit_file = os.path.join(_data, mockname, 'lia', 'bestfit_%s.npy' %(key))
    res = np.load(bestfit_file, allow_pickle=True)
    res = res.tolist()

    text = 'leaf area=%.2f \n %s=%.4f \n %s=%.4f \n %s=%.4f ' %(res['leafsize'], 'voxel_size_w', res['voxel_size_w_bestfit'],'kd3_sr', res['kd3_sr_bestfit'],'max_nn', res['max_nn_bestfit'])
    print(text)

    chi2 = lia.leaf_angle(points, mockname, key, res['voxel_size_w_bestfit'], 
                            res['kd3_sr_bestfit'], res['max_nn_bestfit'], save=True,
                                savefig=True, text=text)
    
    print(chi2)

leaf area=0.04 
 voxel_size_w=0.0100 
 kd3_sr=1.0000 
 max_nn=5.0000 
0.0009380330384670287


## `Leaf Area Density` (LAD)

In [92]:

downsample = 0.02
voxel_size = 0.2
show = False
sample = None

In [93]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['sx', 'sy', 'sz']])

if downsample is not None:

    resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)

    idx = np.random.randint(0, len(df), int(len(df) * downsample))
    inds = np.zeros(len(df), dtype=bool)
    inds[idx] = True

    points = POINTS[inds]
    sensors = SENSORS[inds]

    outdir = os.path.join(resdir, 'inds.npy')
    np.save(outdir, inds)

else:

    resdir = os.path.join(_data, mockname, 'lad')
    if not os.path.exists(resdir):
        os.makedirs(resdir)

if sample is not None:

    idx = np.random.randint(0, len(df), int(sample))
    points = POINTS[idx]
    sensors = SENSORS[idx]

for key, val in trees.items():

    inPR = (val) & (leaves) & (inds)
    pointsPR = POINTS[inPR]
    m3s, m3count= rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)



max --> [44, 43, 29]
min --> [0, 0, 0]


3121it [00:41, 75.84it/s]


tot vox: 	 59400
voxels hitted: 	 40622
Percentage of voxels hitted by beam: 0.68


In [95]:
if downsample is not None:
    inds_file = os.path.join(resdir, 'inds.npy')
    inds = np.load(inds_file)


for key, val in trees.items():

    inPR = (val) & (leaves) & (inds)
    pointsPR = POINTS[inPR]
    sensorsPR = SENSORS[inPR]

    m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)
    # get in down sample boolean array for LPC size
    inds_ = inds[(val) & (leaves)]
    lia, ws = lad.downsample_lia(mockname, key, inds_)
    voxk = lad.get_voxk(pointsPR, voxel_size)
    bia = lad.get_bia(pointsPR, sensorsPR)
    meshfile = lad.get_meshfile(mockname)

    # print(meshfile)
    # print(len(inds), len(inds_), np.sum(inds_))
    # print(len(lia), len(ws))
    # print(len(voxk))
    # print(len(bia))



max --> [44, 43, 29]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (45, 44, 30)
ray tracker voxel dimensions: 	 (45, 44, 30)
Number of voxels ocupied by points cloud: 	 1127
Number of voxels ocupied by beam points cloud: 	 40622
Total number of voxels in plant regions: 	 59400
Number of voxels with attribute 1: 	 1127
Number of voxels with attribute 2: 	 39495
Number of voxels with attribute 3: 	 18778
/Users/omar/projects/planttech/data/test/mesh.ply
157500 68192 1320
1320 1320
1320
1320


# DEV ZONE...